In [5]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

In [6]:
import ast
class allaction():
    
    def __init__(self, file):
        self.df = pd.read_csv(file) 
        self.orderId, self.taskId, self.taskTicks, self.workerId, self.workerTicks = [], [], [], [], []
        self.ResponseId, self.Round, self.Tick = [], [], []
    #taskId: 0~idle, 1~chop, 2~cook, 3~plate
    #workerId: 0~chef, 1~sou-chef, 2~server
    #           0~sou-chef, 1~server, CHANGE!!! 0~idle, 1~chef, 2~sou-chef, 3~server
    
    def parse(self):
        # all id + 1, leaving 0 to be idle
        action = self.df
        for i in np.arange(action.shape[0]):
            row = action.iloc[i,:]
            for act in ["action1", "action2", "action3"]:
                # when there is no action in this act
                if type(row[act]) != str:
                    self.orderId.append(0)
                    self.taskId.append(0)
                    self.taskTicks.append(0)
                    self.workerId.append(0)
                    self.workerTicks.append(0)
                else:
                    a = ast.literal_eval(row[act])
                    self.orderId.append(a["order_id"] + 1)
                    self.taskId.append(a["task_id"] + 1)
                    self.taskTicks.append(a["task_ticks"])
                    self.workerId.append(a["worker_id"] + 1)
                    self.workerTicks.append(a["worker_ticks"])

                self.ResponseId.append(row["ResponseId"])
                self.Round.append(row["round"])
                self.Tick.append(row["tick"])
                
    def get(self):
        self.parse()
        d = {
            "ResponseId": self.ResponseId,
            "round": self.Round,
            "tick": self.Tick,
            "orderId": self.orderId, 
            "taskId": self.taskId, 
            "taskTicks": self.taskTicks, 
            "workerId": self.workerId, 
            "workerTicks": self.workerTicks
        }
        return pd.DataFrame(d)

In [7]:
file =  "../data/phase2/phase2_action.csv"
action = allaction(file).get()
action = action[action["round"] <= 2]
action = action.drop("workerTicks", axis=1)
action

,ResponseId,round,tick,orderId,taskId,taskTicks,workerId
0,R_09f8RCjNyCadwPv,1,1,1,1,2,1
1,R_09f8RCjNyCadwPv,1,1,2,1,2,2
2,R_09f8RCjNyCadwPv,1,1,3,1,2,3
3,R_09f8RCjNyCadwPv,1,2,0,0,0,0
4,R_09f8RCjNyCadwPv,1,2,0,0,0,0
...,...,...,...,...,...,...,...
586474,R_zffMypmlgQF5Yl3,2,18,0,0,0,0
586475,R_zffMypmlgQF5Yl3,2,18,0,0,0,0
586476,R_zffMypmlgQF5Yl3,2,19,4,3,2,3
586477,R_zffMypmlgQF5Yl3,2,19,0,0,0,0


In [172]:
label=pd.read_csv( "../data/phase2/label_after.csv").drop("Unnamed: 0", axis=1)
label

,ResponseId,label,suggested_tip_disrupted
0,R_10HkPUkR6o0qDFT,9,Server cooks twice
1,R_1JCkfcOwq1eNbLu,9,"Make sure the server cooks twice, even if that..."
2,R_1K3WThoBHDDfbk0,9,the server needs to cook twice
3,R_1M40hSm4fF4Saxv,9,Server cooks twice
4,R_1Q5tRaFyI9DwIn4,9,Make the server cook twice
...,...,...,...
285,R_XoEfxfWvLu2dznr,8,good
286,R_2s5D6EJ0FlMyr39,8,0.5
287,R_2Vf01KGZrR5fscp,1,leave some chefs idle
288,R_38EvwAyoLTJRsZP,8,game didn't load


In [175]:
action = action.merge(label,how = "inner",
            left_on="ResponseId", right_on="ResponseId")
y = action["label"]
action = action.iloc[:,0:7]
action

,ResponseId,round,tick,orderId,taskId,taskTicks,workerId
0,R_0HBxBV8U696D9QJ,1,1,1,1,2,1
1,R_0HBxBV8U696D9QJ,1,1,2,1,2,2
2,R_0HBxBV8U696D9QJ,1,1,3,1,2,3
3,R_0HBxBV8U696D9QJ,1,2,0,0,0,0
4,R_0HBxBV8U696D9QJ,1,2,0,0,0,0
...,...,...,...,...,...,...,...
33427,R_zZQIbE0LFD13yRX,2,29,0,0,0,0
33428,R_zZQIbE0LFD13yRX,2,29,0,0,0,0
33429,R_zZQIbE0LFD13yRX,2,30,0,0,0,0
33430,R_zZQIbE0LFD13yRX,2,30,0,0,0,0


In [199]:
y = action.merge(label,how = "inner",
            left_on="ResponseId", right_on="ResponseId").groupby("ResponseId").agg(np.mean)["label"].astype(int)
y

ResponseId
R_0HBxBV8U696D9QJ    2
R_0SZ9OfuWvmqeMqR    6
R_10DdLv5uPLAfpAR    3
R_10Ggl6dEPfo9ipw    1
R_10HkPUkR6o0qDFT    9
                    ..
R_yZJ03FFfe8jfazn    9
R_ykkhkGYu1KpIM8h    1
R_ym9gyf6T2XORxAt    1
R_z8OxWJ4ua6nKKB3    8
R_zZQIbE0LFD13yRX    3
Name: label, Length: 245, dtype: int64

In [177]:
t1 = action.drop("orderId", axis=1)
t2 = t1.merge(pd.get_dummies(t1["taskId"]), left_index=True, right_index=True).drop("taskId", axis=1).rename(
    columns={0:"idle", 1:"chop", 2:"cook", 3:"plate"})
df = t2.merge(pd.get_dummies(t2["workerId"]), left_index=True, right_index=True).drop("workerId", axis=1).rename(
    columns={0:"idle", 1:"chef", 2:"sou-chef", 3:"server"})
df.drop("taskTicks", axis=1).groupby(["ResponseId", "round", "tick"]).sum().iloc[:,1:]

chop  cook  plate  idle  chef  sou-chef  server
ResponseId        round tick                                                 
R_0HBxBV8U696D9QJ 1     1        3     0      0     0     1         1       1
                        2        0     0      0     3     0         0       0
                        3        0     1      0     2     1         0       0
                        4        0     1      0     2     1         0       0
                        5        0     2      0     1     1         1       0
...                            ...   ...    ...   ...   ...       ...     ...
R_zZQIbE0LFD13yRX 2     26       0     0      0     3     0         0       0
                        27       0     0      0     3     0         0       0
                        28       0     0      0     3     0         0       0
                        29       0     0      0     3     0         0       0
                        30       0     0      0     3     0         0       0

[11144 rows x 7 columns]

In [178]:
df

,ResponseId,round,tick,taskTicks,idle,chop,cook,plate,idle,chef,sou-chef,server
0,R_0HBxBV8U696D9QJ,1,1,2,0,1,0,0,0,1,0,0
1,R_0HBxBV8U696D9QJ,1,1,2,0,1,0,0,0,0,1,0
2,R_0HBxBV8U696D9QJ,1,1,2,0,1,0,0,0,0,0,1
3,R_0HBxBV8U696D9QJ,1,2,0,1,0,0,0,1,0,0,0
4,R_0HBxBV8U696D9QJ,1,2,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
33427,R_zZQIbE0LFD13yRX,2,29,0,1,0,0,0,1,0,0,0
33428,R_zZQIbE0LFD13yRX,2,29,0,1,0,0,0,1,0,0,0
33429,R_zZQIbE0LFD13yRX,2,30,0,1,0,0,0,1,0,0,0
33430,R_zZQIbE0LFD13yRX,2,30,0,1,0,0,0,1,0,0,0


In [179]:
a = df.drop("taskTicks", axis=1)
X = a.groupby(["ResponseId", "round", "tick"]).agg(lambda x: x.iloc[0]).merge(
    a.groupby(["ResponseId", "round", "tick"]).agg(lambda x: x.iloc[1]).merge(
        a.groupby(["ResponseId", "round", "tick"]).agg(lambda x: x.iloc[2])
    , left_index=True, right_index=True), left_index=True, right_index=True)

In [180]:
from tqdm import tqdm

In [181]:
# chop0, chef0, server3
embeddings = []
for i in tqdm(np.arange(X.shape[0])):
    embedding = [0,0,0,0,0,0,0,0,0]
    thisline = X.iloc[i,:]
    
    if thisline["chop"] and thisline["chef"]:
        embedding[0] = 1
    elif thisline["chop"] and thisline["sou-chef"]:
        embedding[1] = 1
    elif thisline["chop"] and thisline["server"]:
        embedding[2] = 1 
    elif thisline["cook"] and thisline["chef"]:
        embedding[3] = 1 
    elif thisline["cook"] and thisline["sou-chef"]:
        embedding[4] = 1
    elif thisline["cook"] and thisline["server"]:
        embedding[5] = 1 
    elif thisline["plate"] and thisline["chef"]:
        embedding[6] = 1 
    elif thisline["plate"] and thisline["sou-chef"]:
        embedding[7] = 1
    elif thisline["plate"] and thisline["server"]:
        embedding[8] = 1 
        
    if thisline["chop_x"] and thisline["chef_x"]:
        embedding[0] = 1
    elif thisline["chop_x"] and thisline["sou-chef_x"]:
        embedding[1] = 1
    elif thisline["chop_x"] and thisline["server_x"]:
        embedding[2] = 1 
    elif thisline["cook_x"] and thisline["chef_x"]:
        embedding[3] = 1 
    elif thisline["cook_x"] and thisline["sou-chef_x"]:
        embedding[4] = 1
    elif thisline["cook_x"] and thisline["server_x"]:
        embedding[5] = 1 
    elif thisline["plate_x"] and thisline["chef_x"]:
        embedding[6] = 1 
    elif thisline["plate_x"] and thisline["sou-chef_x"]:
        embedding[7] = 1
    elif thisline["plate_x"] and thisline["server_x"]:
        embedding[8] = 1         
        
    if thisline["chop_y"] and thisline["chef_y"]:
        embedding[0] = 1
    elif thisline["chop_y"] and thisline["sou-chef_y"]:
        embedding[1] = 1
    elif thisline["chop_y"] and thisline["server_y"]:
        embedding[2] = 1 
    elif thisline["cook_y"] and thisline["chef_y"]:
        embedding[3] = 1 
    elif thisline["cook_y"] and thisline["sou-chef_y"]:
        embedding[4] = 1
    elif thisline["cook_y"] and thisline["server_y"]:
        embedding[5] = 1 
    elif thisline["plate_y"] and thisline["chef_y"]:
        embedding[6] = 1 
    elif thisline["plate_y"] and thisline["sou-chef_y"]:
        embedding[7] = 1
    elif thisline["plate_y"] and thisline["server_y"]:
        embedding[8] = 1  
    
    embeddings.append(embedding)
   

100%|██████████| 11144/11144 [00:04<00:00, 2569.17it/s]


In [228]:
kkk = df.drop("taskTicks", axis=1).groupby(["ResponseId", "round", "tick"]).sum().iloc[:,1:]
kkk["index"] = np.arange(kkk.shape[0])
X = kkk.merge(pd.DataFrame(np.array(embeddings)),how="left", left_on="index", right_index=True).drop(
    "chop", axis=1).drop("cook", axis=1).drop("plate", axis=1).drop("idle", axis=1).drop(
    "chef", axis=1).drop("sou-chef", axis=1).drop("server", axis=1).drop("index", axis=1)
X = X.reset_index()
X

,ResponseId,round,tick,0,1,2,3,4,5,6,7,8
0,R_0HBxBV8U696D9QJ,1,1,1,1,1,0,0,0,0,0,0
1,R_0HBxBV8U696D9QJ,1,2,0,0,0,0,0,0,0,0,0
2,R_0HBxBV8U696D9QJ,1,3,0,0,0,1,0,0,0,0,0
3,R_0HBxBV8U696D9QJ,1,4,0,0,0,1,0,0,0,0,0
4,R_0HBxBV8U696D9QJ,1,5,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11139,R_zZQIbE0LFD13yRX,2,26,0,0,0,0,0,0,0,0,0
11140,R_zZQIbE0LFD13yRX,2,27,0,0,0,0,0,0,0,0,0
11141,R_zZQIbE0LFD13yRX,2,28,0,0,0,0,0,0,0,0,0
11142,R_zZQIbE0LFD13yRX,2,29,0,0,0,0,0,0,0,0,0


In [232]:
max_time_tick = max(X.groupby("ResponseId").count()["round"])

In [234]:
data = []
for player in tqdm(set(X["ResponseId"])):
    batch = np.array(X[X["ResponseId"] == player].iloc[:,3:]).tolist()
    padding = max_time_tick - len(batch)
    if padding > 0  :
        for _ in np.arange(padding):
            batch.append([0,0,0,0,0,0,0,0,0])
    data.append(batch)

100%|██████████| 245/245 [00:00<00:00, 792.14it/s]


In [238]:
np.array(data).shape

(245, 83, 9)

In [208]:
from torch import nn

In [207]:
class RNNModel(nn.Module):
    """循环神经网络模型"""
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.num_hiddens = self.rnn.hidden_size
        # 如果RNN是双向的（之后将介绍），num_directions应该是2，否则应该是1
        if not self.rnn.bidirectional:
            self.num_directions = 1
            self.linear = nn.Linear(self.num_hiddens, self.vocab_size)
        else:
            self.num_directions = 2
            self.linear = nn.Linear(self.num_hiddens * 2, self.vocab_size)

    def forward(self, inputs, state):
        X = F.one_hot(inputs.T.long(), self.vocab_size)
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)
        # 全连接层首先将Y的形状改为(时间步数*批量大小,隐藏单元数)
        # 它的输出形状是(时间步数*批量大小,词表大小)。
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, state

    def begin_state(self, device, batch_size=1):
        if not isinstance(self.rnn, nn.LSTM):
            # nn.GRU以张量作为隐状态
            return  torch.zeros((self.num_directions * self.rnn.num_layers,
                                 batch_size, self.num_hiddens),
                                device=device)
        else:
            # nn.LSTM以元组作为隐状态
            return (torch.zeros((
                self.num_directions * self.rnn.num_layers,
                batch_size, self.num_hiddens), device=device),
                    torch.zeros((
                        self.num_directions * self.rnn.num_layers,
                        batch_size, self.num_hiddens), device=device))

In [214]:
num_hiddens = 256
rnn_layer = nn.RNN(9, num_hiddens)
net = RNNModel(rnn_layer, vocab_size=9)

In [215]:
state = torch.zeros((1, 245, num_hiddens))
state.shape

torch.Size([1, 245, 256])

In [217]:
X = torch.rand(size=(35, 245, 9))
Y, state_new = rnn_layer(X, state)
Y.shape, state_new.shape

(torch.Size([35, 245, 256]), torch.Size([1, 245, 256]))

In [247]:

df.sample(frac=1, random_state=42).reset_index().drop("index", axis=1)

,ResponseId,round,tick,taskTicks,idle,chop,cook,plate,idle,chef,sou-chef,server
0,R_szXwz0MKupI9DVL,1,2,0,1,0,0,0,1,0,0,0
1,R_3jYYDNFziT9HLhy,1,12,0,1,0,0,0,1,0,0,0
2,R_8Ic3EXtG9WooqpX,1,12,0,1,0,0,0,1,0,0,0
3,R_25HGKC7lGzezQXZ,1,17,0,1,0,0,0,1,0,0,0
4,R_3DwDLvNoR71bC9Q,2,24,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
33427,R_2wS5obwKA8wgUWW,2,9,0,1,0,0,0,1,0,0,0
33428,R_1mf7bqHoccP7ipz,2,1,2,0,1,0,0,0,0,1,0
33429,R_2TOcvvA7vXvY0c0,2,12,0,1,0,0,0,1,0,0,0
33430,R_10uOrh9zAwHCC7P,2,3,0,1,0,0,0,1,0,0,0
